In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
CREATE OR REPLACE TABLE DOCUMENTS_RAW_PARSED AS
SELECT 
    RELATIVE_PATH AS FILENAME,
    FILE_URL,
    SIZE,
    LAST_MODIFIED,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
            '@documents',
             relative_path,
             {'mode': 'OCR'}
        )
    ) AS PARSED_CONTENT
FROM 
    DIRECTORY(@documents)
WHERE 
    1 = 1 
    --AND RELATIVE_PATH = 'Briefing - proactive-disclosure-2024-12-12-secu-borders-en.pdf'

    

In [ ]:
SELECT * FROM DOCUMENTS_RAW_PARSED;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    'I will provide you with a list of questions that contract management representatives have about legal contracts. The idea is to extract the relevant entities so I can use them for faster retrieval in a system that is holding the content of these documents. Do not use any formatting, just provide entities/terms using a colon to separate them. Have each key and value be on a new line.

    See below the questions:
    
    Who are the hotel owners for each contract?
What are the contract terms? (Compare results w/Q3)
What are the key terms of the contract? (Compare results w/Q2)
What is the effective date of each contract?
Does the contract include any Third Party Beneficieries?
What are the termination terms, Performance Test termination right and termination on sale in the contract?
Please review all files in this repository and return a list of file names that are management agreements or hotel services agreements.
Find all management and franchise agreements that contain a Force Majeure provision
Does this contract require Owner to provide information on financings?
Review all of the documents and tell me their renewal date.
What changed between the Management Agreement and Amendment 1 for [HOTEL NAME}?
Who are the hotel owners for each contract? Please put the output in a table, showing the contract name.
Please review all contracts and put the information in a table in alphabetical order
Show me all clauses that refer to exclusivity.
What is the difference between the Management Agreement for [HOTEL X] and [HOTEL Y]?
Please review all documents and create a table with all of the references to management fee and renewal terms.
Please review all management and franchise agreements for exclusivity language with a termination date
What are the differences between the Management Agreement for [HOTEL X] and all amendments for the same hotel?
What are the Performance tests in each management agreement?  And are there any Performance Test termination rights?
Following the previous question: summarize the differences in performance Tests for [HOTEL 1] and [HOTEL 2].
Please review all of the files in this repository and pull out any effective dates by hotel name and contract type.
Please review every file in this repository for clauses for exclusivity and put the information into a table for my review.
Please summarize the franchise agreement clauses that are the most important.
Find all contracts with Key Money and create a table listing the contract name, hotel name and amount of key money in order of value with the highest value of Key Money at the top of the list. 
Find all contracts that are related to a Franchise agreement and report the current Fee Structure for each contract.
Review each document for the following list of fields and return a table with all of the data: Brand, Hotel Country, Operating Terms, Contract Effective Date, Expiration Date, Renewal Rights, Termination on Sale, Key Money 
what are the renewal terms, who has the right to renew, and is there a date the extension notice needs to be sent?
Find all managements agreements that do not have a renewal provision and put them in a list with the expiration dates in order of expiration'
)

In [ ]:
CREATE OR REPLACE TABLE DOCUMENTS_RAW_PARSED_METADATA AS (
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    'I will provide a list of key terms / entitites that I need you to find the corresponding value in the document provided. This will be indexed by a retrieval system containing many similar documents. Follow these instructions:

            1. Do not dwell on low level details. Only provide key high level information that a human might be expected to provide when searching for this doc.
    
        2. Do not use any formatting, just provide keys and values using a colon to separate key and value. Have each key and value be on a new line. 

        3. If there is a missing value just indicate N/A
    
    List of Entities:
    hotel owners
contract terms
key terms
effective date
third party beneficiaries
termination terms
performance test termination right
termination on sale
management agreements
hotel services agreements
franchise agreements
force majeure provision
owner financing requirements
renewal date
amendment changes
exclusivity clauses
management fee
performance tests
performance test termination rights
contract type
key money
fee structure
brand
hotel country
operating terms
contract effective date
expiration date
renewal rights
renewal terms
renewal notice date
hotel name
contract name
termination date
exclusivity language
franchise agreement clauses
'|| '\n\nDoc starts here:\n' 
        || PARSED_CONTENT
        || '\nDoc ends here\n\n'
) DOCUMENT_METADATA
FROM 
    DOCUMENTS_RAW_PARSED )

In [ ]:
SELECT * FROM DOCUMENTS_RAW_PARSED_METADATA

In [ ]:
-- CREATE CORTEX SEARCH SERVICE FOR METADATA
CREATE OR REPLACE CORTEX SEARCH SERVICE CS_DOCUMENTS_METADATA
ON CONTENT_METADATA 
ATTRIBUTES FILENAME
WAREHOUSE = DEMO_WH 
TARGET_LAG = '1 minute' 
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS ( SELECT * FROM DOCUMENTS_RAW_PARSED_METADATA ); 

In [ ]:
-- STEP 2: GENERATE CHUNKS AND PREPEND CONTEXT TO CHUNKb

CREATE OR REPLACE TABLE CHUNKS_CONTEXTUALIZED AS (
    WITH SPLIT_TEXT_CHUNKS AS (
        SELECT
            FILENAME,
            FILE_URL,
            C.VALUE AS CHUNK,
        FROM
           DOCUMENTS_RAW_PARSED_METADATA,
           LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              PARSED_CONTENT,
              'none',
              1800, -- SET CHUNK SIZE
              300 -- SET CHUNK OVERLAP
           )) C
    )
    SELECT
        M.FILENAME,
        M.FILE_URL,
        CONCAT(M.CONTENT_METADATA, '\n\n', C.CHUNK) AS CONTEXTUALIZED_CHUNK,
    FROM
        SPLIT_TEXT_CHUNKS C
    JOIN
        DOCUMENTS_RAW_PARSED_METADATA M ON C.FILENAME = M.FILENAME
);

In [ ]:
SELECT * FROM CHUNKS_CONTEXTUALIZED;

In [ ]:
-- CREATE CORTEX SEARCH SERVICE FOR CHUNKS
CREATE OR REPLACE CORTEX SEARCH SERVICE CS_DOCUMENTS_CHUNKS
ON CONTEXTUALIZED_CHUNK 
ATTRIBUTES FILENAME
WAREHOUSE = DEMO_WH 
TARGET_LAG = '1 minute' 
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS ( SELECT * FROM CHUNKS_CONTEXTUALIZED     ); 